In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import csv

# เปิด Chrome WebDriver
driver = webdriver.Chrome()
driver.get("https://arxiv.org/search/")

time.sleep(3)

# ค้นหาฟิลด์ค้นหาบนหน้า
search_field = driver.find_element(By.ID, "query")
search_field.send_keys("engineering")

# คลิกปุ่มค้นหา
button = driver.find_element(By.XPATH, '//*[@class="button is-link is-medium"]')
driver.execute_script("arguments[0].click();", button)

time.sleep(3)  # รอให้หน้าโหลดข้อมูลเสร็จ

dropdown = driver.find_element(By.ID, "size")

# สร้างตัวเลือกจาก Select
select = Select(dropdown)

# เลือกตัวเลือกที่มีค่า value เป็น 200
select.select_by_value("200")

button = driver.find_element(By.XPATH, '//*[@class="button is-small is-link"]')
driver.execute_script("arguments[0].click();", button)

# ดึง HTML ของหน้า
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# ค้นหาทุก <li> ที่มี class เป็น 'arxiv-result'
articles = soup.find_all('li', class_='arxiv-result')

article_data = []
for article in articles:
    # ดึง title
    title = article.find('p', class_='title is-5 mathjax').get_text(strip=True)
    
    # ดึง authors และแปลงชื่อให้เป็น Lastname Firstname
    authors = article.find('p', class_='authors').get_text(strip=True).replace("Authors:", "").strip()
    author_list = authors.split(",")  # แยกชื่อออกตามเครื่องหมายจุลภาค
    formatted_authors = []
    
    for author in author_list:
        name_parts = author.split(' ')
        if len(name_parts) == 2:  # ถ้ามีชื่อและนามสกุล
            last_name = name_parts[0].strip()
            first_name = name_parts[1].strip()
            if first_name:
                formatted_authors.append(f"{last_name} {first_name[0]}.")  # ใช้แค่ตัวอักษรแรกของชื่อ
            else:
                formatted_authors.append(f"{last_name}")  # ถ้าไม่มีชื่อ
        else:  # ถ้าชื่อมีแค่ชื่อนามสกุล
            formatted_authors.append(f"{name_parts[0].strip()}")
    
    # เชื่อมผู้เขียนทั้งหมด
    formatted_authors = "; ".join(formatted_authors)
    
    # ดึง abstract
    abstract = article.find('span', class_='abstract-short').get_text(strip=True)

    # ดึง publication date (ดึงจากข้อความที่มีคำว่า 'Submitted')
    pub_date_section = article.find('p', class_='is-size-7')
    pub_date = "No date found"  # กำหนดค่าเริ่มต้น
    if pub_date_section:
        pub_date_text = pub_date_section.get_text(strip=True)
        pub_date_text = pub_date_text.split(';')
        pub_date_text = pub_date_text[0].replace("Submitted","").strip()

        parts = pub_date_text.split(" " or ", ")


            
            # แปลงเดือนเป็นตัวเลข
        month_dict = {
            "January,": '01', "February,": '02', "March,": '03', "April,": '04', "May,": '05',
            "June,": '06', "July,": '07', "August,": '08', "September,": '09', "October,": '10',
            "November,": '11', "December,": '12'
        }
        month_number = month_dict.get(parts[1], '01')  # ถ้าไม่เจอเดือน จะใช้ '01' เป็นค่าเริ่มต้น

            # สร้างวันที่ในรูปแบบ YYYY-MM-DD
        pub_date = f"{parts[2]}-{month_number}-{parts[0].zfill(2)}"
    

    # เพิ่มข้อมูลที่ดึงมาใน list
    article_data.append({
        'title': title,
        'authors': formatted_authors,
        'abstract': abstract,
        'publication_date': pub_date
    })

# เปิดไฟล์ CSV สำหรับการเขียนข้อมูล
with open('../../ExtractedData/arxiv_articles_2024.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['Title', 'Authors', 'Abstract', 'Publication_Date'])
    
    # เขียนชื่อคอลัมน์
    writer.writeheader()

    # เพิ่มข้อมูลที่ดึงมาในไฟล์ CSV
    for data in article_data:
        writer.writerow({
            'Title': data['title'],
            'Authors': data['authors'],
            'Abstract': data['abstract'],
            'Publication_Date': data['publication_date']
        })

print("Data has been saved to arxiv_articles_2024.csv")


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="query"]"}
  (Session info: chrome=131.0.6778.109); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104b3bb08 cxxbridge1$str$ptr + 3651580
1   chromedriver                        0x0000000104b34358 cxxbridge1$str$ptr + 3620940
2   chromedriver                        0x000000010459c4b4 cxxbridge1$string$len + 89224
3   chromedriver                        0x00000001045e0898 cxxbridge1$string$len + 368748
4   chromedriver                        0x000000010461a0fc cxxbridge1$string$len + 604368
5   chromedriver                        0x00000001045d50b0 cxxbridge1$string$len + 321668
6   chromedriver                        0x00000001045d5d00 cxxbridge1$string$len + 324820
7   chromedriver                        0x0000000